# All in One Image Dehazing using DL
Github Link: https://github.com/tusharsircar95/All-In-One-Image-Dehazing-Tensorflow

## Main and Defined Functions:

In [1]:
# import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.layers import *
# import glob
# import random
import cv2
import time
# !pip install imutils
import imutils
from imutils.video import VideoStream
import tensorflow.compat.v1 as tf
from PIL import Image
import matplotlib.pyplot as plt
%load_ext autotime


n_epochs = 3
batch_size = 8
learning_rate = 1e-4
weight_decay = 1e-4

def haze_net(X):
  
  conv1 = Conv2D(3,1,1,padding="SAME",activation="relu",use_bias=True,kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(X)
  conv2 = Conv2D(3,3,1,padding="SAME",activation="relu",use_bias=True,kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(conv1)
  concat1 = tf.concat([conv1,conv2],axis=-1)
  
  conv3 = Conv2D(3,5,1,padding="SAME",activation="relu",use_bias=True,kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(concat1)
  concat2 = tf.concat([conv2,conv3],axis=-1)
  
  conv4 = Conv2D(3,7,1,padding="SAME",activation="relu",use_bias=True,kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(concat2)
  concat3 = tf.concat([conv1,conv2,conv3,conv4],axis=-1)
  
  conv5 = Conv2D(3,3,1,padding="SAME",activation="relu",use_bias=True,kernel_initializer=tf.initializers.random_normal(stddev=0.02),
                kernel_regularizer=tf.keras.regularizers.l2(weight_decay))(concat3)
  K = conv5
  
  output = ReLU(max_value=1.0)(tf.math.multiply(K,X) - K + 1.0)
  #output = output / 255.0
  
  return output

def setup_data_paths(orig_images_path,hazy_images_path):
  
  orig_image_paths = glob.glob(orig_images_path + "/*.jpg")
  n = len(orig_image_paths) 
  random.shuffle(orig_image_paths)
  
  train_keys = orig_image_paths[:int(0.90*n)]
  val_keys = orig_image_paths[int(0.90*n):]
    
  split_dict = {}
  for key in train_keys:
    split_dict[key] = 'train'
  for key in val_keys:
    split_dict[key] = 'val'
  
  train_data = []
  val_data = []
  
  hazy_image_paths = glob.glob(hazy_images_path + "/*.jpg")
  for path in hazy_image_paths:
    label = path.split('/')[-1]
    orig_path = orig_images_path + "/" + label.split('_')[0] + '_' + label.split('_')[1] + ".jpg"
    if(split_dict[orig_path] == 'train'):
      train_data.append([path,orig_path])
    else: val_data.append([path,orig_path])
  
  return train_data, val_data
    
def load_image(X):
  X = tf.io.read_file(X)
  X = tf.image.decode_jpeg(X,channels=3)
  X = tf.image.resize(X,(480,640))
  X = X / 255.0
  return X

def showImage(x):
  x = np.asarray(x*255,dtype=np.int32)
  plt.figure()
  plt.imshow(x)
  plt.show()
    
def create_datasets(train_data,val_data,batch_size):
  
  train_ds_hazy = tf.data.Dataset.from_tensor_slices([data[0] for data in train_data]).map(lambda x: load_image(x))
  train_ds_orig = tf.data.Dataset.from_tensor_slices([data[1] for data in train_data]).map(lambda x: load_image(x))
  train_ds = tf.data.Dataset.zip((train_ds_hazy,train_ds_orig)).shuffle(100).repeat().batch(batch_size)

  val_ds_hazy = tf.data.Dataset.from_tensor_slices([data[0] for data in val_data]).map(lambda x: load_image(x))
  val_ds_orig = tf.data.Dataset.from_tensor_slices([data[1] for data in val_data]).map(lambda x: load_image(x))
  val_ds = tf.data.Dataset.zip((val_ds_hazy,val_ds_orig)).shuffle(100).repeat().batch(batch_size)

  iterator = tf.data.Iterator.from_structure(train_ds.output_types,train_ds.output_shapes)
  
  train_init_op = iterator.make_initializer(train_ds)
  val_init_op = iterator.make_initializer(val_ds)
  
  return train_init_op, val_init_op, iterator

tf.disable_eager_execution()
X = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
Y = tf.placeholder(shape=(None,480,640,3),dtype=tf.float32)
dehazed_X = haze_net(X)

def dehaze_image(image):
  # print(image.shape)
  image = image.reshape((1,) + image.shape)
  dehazed_image = sess.run(dehazed_X,feed_dict={X:image,Y:image})
#   RGB_img = cv2.cvtColor(dehazed_image[0], cv2.COLOR_BGR2RGB)
  return dehazed_image[0]*255

time: 2.78 s (started: 2022-12-15 18:38:04 +05:30)


## Single Image Dehazing Code:

In [4]:
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess,r'C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt')
    test_image_paths = [r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\From Sir\dataset\DSC_3074.jpg"]

    for path in test_image_paths:
        # image_label = path.split(test_input_folder)[-1][1:]
        image = Image.open(path)
        w, h = image.size
#         print(type(image))
        image = image.resize((640, 480))
        image = np.asarray(image) / 255.0
        image = image.reshape((1,) + image.shape)
        dehazed_image = sess.run(dehazed_X,feed_dict={X:image,Y:image})
        
        RGB_img = cv2.cvtColor(dehazed_image[0], cv2.COLOR_BGR2RGB)
        RGB_img = cv2.resize(RGB_img, (w,h))
        # Show Image:
#         print(RGB_img.shape, RGB_img.dtype)
        cv2.imshow('img',RGB_img)
        
        # Saving the Image:
#         cv2.imwrite(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Images\Online Images\img.png',RGB_img*255)
        
        cv2.waitKey(0) & 0xFF == ord('q')
        cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt


## Video Dehazing Code:

In [18]:
tf.disable_eager_execution()
inputVideoPath = r"E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\Vdo_3.mp4"
vidObj = cv2.VideoCapture(inputVideoPath)
saver = tf.train.Saver()

# Loop for frames:
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
frame_width = vidObj.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = vidObj.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = vidObj.get(cv2.CAP_PROP_FPS)
frame_nos = vidObj.get(cv2.CAP_PROP_FRAME_COUNT)
# save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output_small.mp4',fourcc, fps, (640, 480))
save = cv2.VideoWriter(r'E:\College Education\DA-IICT\Study\Sem 3\Pratik - Projects\Video Dehazing\Input Images & Videos\Input Videos\output_small.mp4',fourcc, fps, (int(frame_width), int(frame_height)))

###
for frame_idx in range(int(frame_nos+1)):
  ret, I = vidObj.read()

  if ret:
    I = cv2.resize(I, (640, 480))
    x = I / 255.0
    with tf.Session() as sess:
      saver.restore(sess,r'C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt')
      x = dehaze_image(x)
    # x = x * 255
    x = (np.rint(x)).astype(np.uint8)
#     x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = cv2.resize(x, (int(frame_width), int(frame_height)))
    save.write(x)
    
  # Breaking Loop:
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

vidObj.release()
save.release()
cv2.destroyAllWindows()

print('Execution Completed')

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

...

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

## Real-Time Dehazing Code:

In [ ]:
if __name__ == "__main__":

    vidObj_vs = VideoStream(1).start()
    
    time.sleep(1.0)
    start_time = time.time()
    saver = tf.train.Saver()
    
    # Defining FPS:
    FPS_Start_Time = 0
    FPS = 0
    
    while True:

        I = vidObj_vs.read()
        # FPS:
        FPS_End_Time = time.time()
        FPS = 1/(FPS_End_Time - FPS_Start_Time)
        FPS_Start_Time = FPS_End_Time
        FPS_Text = "FPS: {:.2f}".format(FPS)
        cv2.putText(I, FPS_Text, (5,20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,0), 1)
        
        cv2.imshow('Input Sequence',I)
        
#         I = cv2.resize(I, (640, 480))
        x = I / 255.0
        with tf.Session() as sess:
            saver.restore(sess,r'C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt')
            x = dehaze_image(x)
            x = (np.rint(x)).astype(np.uint8)
#             x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
            cv2.imshow('Output Sequence',x)

        # Breaking Loop:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    vidObj_vs.stream.release()
    # save.release()
    cv2.destroyAllWindows()

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

...

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Notebooks\Video Deazing Project\Github Repos\All-In-One-Image-Dehazing-Tensorflow-master\models\model_checkpoint_7.ckpt
INFO:tensorflow:Restoring parameters from C:\Users\Pratik\Jupyter Note

## Below code is for Eeal-Time reference only:

In [1]:
import cv2
import time

# 0 is default webcam. Change to switch between multiple cameras or IP address.
vc = cv2.VideoCapture(0) 

start_time = time.time()
# FPS update time in seconds
display_time = 2
fc = 0
FPS = 0

while True:

	_, frame = vc.read()

	# OpenCV reads images in BGR color format by default. 
	#frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

	fc+=1
	TIME = time.time() - start_time

	if (TIME) >= display_time :
		FPS = fc / (TIME)
		fc = 0
		start_time = time.time()

	fps_disp = "FPS: "+str(FPS)[:5]
	
	# Add FPS count on frame
	image = cv2.putText(frame, fps_disp, (10, 25),
		cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

	# imshow converts BGR to RGB while saving or displaying.
	cv2.imshow('Video Stream w/ FPS', image)
	key = cv2.waitKey(1) & 0xFF

 	# press q to quit streaming
	if key == ord("q"):
		break
vc.release()
cv2.destroyAllWindows()

In [2]:
# importing required libraries
import cv2  # OpenCV library 
import time # time library
from threading import Thread # library for multi-threading

# defining a helper class for implementing multi-threading 
class WebcamStream :
    # initialization method 
    def __init__(self, stream_id=0):
        self.stream_id = stream_id # default is 0 for main camera 
        
        # opening video capture stream 
        self.vcap      = cv2.VideoCapture(self.stream_id)
        if self.vcap.isOpened() is False :
            print("[Exiting]: Error accessing webcam stream.")
            exit(0)
        fps_input_stream = int(self.vcap.get(5)) # hardware fps
            
        # reading a single frame from vcap stream for initializing 
        self.grabbed , self.frame = self.vcap.read()
        if self.grabbed is False :
            print('[Exiting] No more frames to read')
            exit(0)
        # self.stopped is initialized to False 
        self.stopped = True
        # thread instantiation  
        self.t = Thread(target=self.update, args=())
        self.t.daemon = True # daemon threads run in background 
        
    # method to start thread 
    def start(self):
        self.stopped = False
        self.t.start()
    # method passed to thread to read next available frame  
    def update(self):
        while True :
            if self.stopped is True :
                break
            self.grabbed , self.frame = self.vcap.read()
            if self.grabbed is False :
                print('[Exiting] No more frames to read')
                self.stopped = True
                break 
        self.vcap.release()
    # method to return latest read frame 
    def read(self):
        return self.frame
    # method to stop reading frames 
    def stop(self):
        self.stopped = True

        
# initializing and starting multi-threaded webcam input stream 
webcam_stream = WebcamStream(stream_id=0) # 0 id for main camera
webcam_stream.start()
# processing frames in input stream

while True :
    if webcam_stream.stopped is True :
        break
    else :
        frame = webcam_stream.read()

    # displaying frame 
    cv2.imshow('frame' , frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

webcam_stream.stop() # stop the webcam stream
cv2.destroyAllWindows()